In [2]:
# import libraries
import pandas as pd 
from itertools import product
import numpy as np
import math

A= ['carrier', 'origin', 'destination'] 
M= ['arrivaldelay', 'departurdelay', 'weatherdelay']
F= ['SUM', 'AVG', 'MIN', 'MAX', 'STD']

df = pd.DataFrame(list(product(A,M,F)), columns=['Attributes', 'Meassure', 'Function'])
df['Utility'] = np.random.uniform(0, 0.5, df.shape[0])
df.head()

,Attributes,Meassure,Function,Utility
0,carrier,arrivaldelay,SUM,0.094380
1,carrier,arrivaldelay,AVG,0.237981
2,carrier,arrivaldelay,MIN,0.240287
3,carrier,arrivaldelay,MAX,0.171546
4,carrier,arrivaldelay,STD,0.380604


In [53]:
def diversity(a,b):
    la = list(a)
    lb = list(b)
    v1 = get_value_d(la[0],lb[0])
    v2 = get_value_d(la[1],lb[1])
    v3 = get_value_d(la[2],lb[2])
    res = float((((v1*3)+(v2*2)+(v3*1))/3)/2)
    return res
    
def get_value_d(attr1, attr2):
    if (attr1) == (attr2):
        d = 0.0
    else:
        d = 1.0
    return d

def most_distant_two_views(data):
    max_distance = 0
    result = []
    for i in range(0,len(data)):
        for j in range(0,len(data)):
            d = diversity(data[i],data[j])
            if d > max_distance:
                max_distance = d
                result = [data[i],data[j]]
    return result

def div_compute(item, S_list):
    S_new = S_list.copy()
    S_new.append(item)
    k = len(S_new)
    new_series = pd.Series(S_new)
    series_set = new_series.apply(lambda row: set(row))
    new_df = series_set.apply(lambda a: series_set.apply(lambda b: diversity(a,b)))
    new_df['tot'] = new_df.sum(axis=1)
    div = (sum(new_df['tot'])/2)/(k*(k-1))
    return div

def max_compute(div, tradeoff=0.9):
    #max_i = math.sqrt(2)
    util = 0.860600865818768
    util = util
    objf = ((1-tradeoff)*util)+(tradeoff*div)
    return objf

def min_compute(div, tradeoff=0.9):
    util = 0.0
    objf = ((1-tradeoff)*util)+(tradeoff*div)
    return objf

# Heart disease X = 68
# hypothetical 0.8 = 12, 0.9 = 38 0.7 = 0, 1.0 = 47
# Static max bound 0.9 = 2, 1.0 = 47

#Pi 80 0.7 = 2, 0.8 = 16, 0.9 = 38, 1.0 = 47

# First Iteration with Actual Bound

In [54]:
xl = pd.ExcelFile("disease.xlsx")
df = xl.parse("Sheet1", header=0)

In [55]:
max_util = max(df['Utility'])
max_util

0.86060086581876793

In [56]:
df_greedy = df.drop(['Utility'],axis=1)
dataset = df_greedy.reset_index(drop=True)
data = dataset.values.tolist()
X = data.copy()
S = most_distant_two_views(data)
X.remove(S[0])
X.remove(S[1])
i = len(S)

In [57]:
X[0] #Example of X

['exang', 'age', 'max']

In [58]:
S # two most distant views

[['exang', 'age', 'sum'], ['fbs', 'chol', 'variance']]

In [59]:
#Creating dataframe of X 
x_df = pd.DataFrame(X, columns=['Attributes', 'Meassure', 'Function']) 
# Calculate diversity value of each X to current set S
x_df['div'] = x_df[['Attributes', 'Meassure', 'Function']].apply(lambda row: div_compute(row,S), axis=1)
# Calculate max and min values of each X
x_df['Umax'] = x_df['div'].apply(lambda row: max_compute(row))
x_df['Umin'] = x_df['div'].apply(lambda row: min_compute(row))
x_df.head(8)

,Attributes,Meassure,Function,div,Umax,Umin
0,exang,age,max,0.500000,0.53606,0.450
1,exang,age,variance,0.416667,0.46106,0.375
2,exang,age,avg,0.500000,0.53606,0.450
3,exang,age,stddev,0.416667,0.46106,0.375
4,fbs,age,variance,0.333333,0.38606,0.300
5,fbs,age,sum,0.472222,0.51106,0.425
6,fbs,age,stddev,0.416667,0.46106,0.375
7,fbs,age,max,0.416667,0.46106,0.375


In [60]:
# is there any rows that minimum value of MAX less than maximum value of MIN? 

In [61]:
# Minimum value of Max
min(x_df['Umax'])

0.38606008658187679

In [62]:
# Maximum value of Min
max(x_df['Umin'])

0.45000000000000001

In [63]:
new_df = x_df[x_df['Umax'] < max(x_df['Umin'])]
len(new_df)

16

In [64]:
# Minimum and Maximum of diversity value 
min(x_df['div']), max(x_df['div'])

(0.33333333333333331, 0.5)

# Superstore Dataset

In [65]:
xl = pd.ExcelFile("superstore_results1.xlsx")
df = xl.parse("Sheet1", header=0)


In [66]:
max_util = max(df['Utility'])
max_util

1.4805322859916039

In [67]:
df_greedy = df.drop(['Utility'],axis=1)
dataset = df_greedy.reset_index(drop=True)
data = dataset.values.tolist()
X = data.copy()
S = most_distant_two_views(data)
X.remove(S[0])
X.remove(S[1])
i = len(S)

In [230]:
X[0] #Example of X

['postal_code', 'profit', 'avg']

In [231]:
S # two most distant views

[['order_date', 'profit', 'avg'], ['ship_mode', 'sales', 'sum']]

In [232]:
#Creating dataframe of X 
x_df = pd.DataFrame(X, columns=['Attributes', 'Meassure', 'Function']) 
# Calculate diversity value of each X to current set S
x_df['div'] = x_df[['Attributes', 'Meassure', 'Function']].apply(lambda row: div_compute(row,S), axis=1)
# Calculate max and min values of each X
x_df['max'] = x_df['div'].apply(lambda row: max_compute(row))
x_df['min'] = x_df['div'].apply(lambda row: min_compute(row))
x_df.head(8)

,Attributes,Meassure,Function,div,max,min
0,postal_code,profit,avg,0.361111,0.379523,0.325
1,order_date,profit,sum,0.333333,0.354523,0.300
2,ship_date,profit,sum,0.416667,0.429523,0.375
3,customer_name,profit,sum,0.416667,0.429523,0.375
4,ship_date,profit,avg,0.416667,0.429523,0.375
5,customer_name,profit,avg,0.416667,0.429523,0.375
6,city,profit,avg,0.416667,0.429523,0.375
7,product_name,profit,avg,0.361111,0.379523,0.325


In [233]:
# is there any rows that minimum value of MAX less than maximum value of MIN? 

In [234]:
# Minimum value of Max
min(x_df['max'])

0.3545228091552402

In [235]:
# Maximum value of Min
max(x_df['min'])

0.45000000000000001

In [236]:
new_df = x_df[x_df['max'] < max(x_df['min'])]
len(new_df)

49

In [237]:
# Minimum and Maximum of diversity value 
min(x_df['div']), max(x_df['div'])

(0.33333333333333331, 0.5)

# Heart Dataset

In [427]:
xl = pd.ExcelFile("heart_results1.xlsx")
df = xl.parse("Sheet1", header=0)


In [428]:
max_util = max(df['Utility'])
max_util

0.54522809155240204

In [429]:
df_greedy = df.drop(['Utility'],axis=1)
dataset = df_greedy.reset_index(drop=True)
data = dataset.values.tolist()
X = data.copy()
S = most_distant_two_views(data)
X.remove(S[0])
X.remove(S[1])
i = len(S)

In [430]:
X[0] #Example of X

['restecg', 'oldpeak', 'max']

In [431]:
S # two most distant views

[['cp', 'oldpeak', 'avg'], ['restecg', 'chol', 'max']]

In [432]:
#Creating dataframe of X 
x_df = pd.DataFrame(X, columns=['Attributes', 'Meassure', 'Function']) 
# Calculate diversity value of each X to current set S
x_df['div'] = x_df[['Attributes', 'Meassure', 'Function']].apply(lambda row: div_compute(row,S), axis=1)
# Calculate max and min values of each X
x_df['max'] = x_df['div'].apply(lambda row: max_compute(row))
x_df['min'] = x_df['div'].apply(lambda row: min_compute(row))
x_df.head(8)

,Attributes,Meassure,Function,div,max,min
0,restecg,oldpeak,max,0.333333,0.333333,0.333333
1,cp,oldpeak,max,0.472222,0.472222,0.472222
2,sex,oldpeak,max,0.472222,0.472222,0.472222
3,exang,oldpeak,avg,0.416667,0.416667,0.416667
4,thal,oldpeak,avg,0.361111,0.361111,0.361111
5,thal,oldpeak,max,0.472222,0.472222,0.472222
6,restecg,oldpeak,avg,0.361111,0.361111,0.361111
7,restecg,chol,avg,0.361111,0.361111,0.361111


In [433]:
# is there any rows that minimum value of MAX less than maximum value of MIN? 

In [434]:
# Minimum value of Max
min(x_df['max'])

0.33333333333333331

In [435]:
# Maximum value of Min
max(x_df['min'])

0.5

In [436]:
new_df = x_df[x_df['max'] < max(x_df['min'])]
len(new_df)

47

In [412]:
# Minimum and Maximum of diversity value 
min(x_df['div']), max(x_df['div'])

(0.33333333333333331, 0.5)

In [413]:
len(X)

68